In [ ]:
#
import pandas as pd
import numpy as np
from numpy import pad
import pickle, os
from concurrent.futures import ThreadPoolExecutor
import time, json
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from voyageai import get_embeddings
import google.generativeai as genai
import transformers
import torch
from openai import OpenAI
from huggingface_hub import login
import openai
from sentence_transformers import SentenceTransformer
import voyageai


In [ ]:
%run config.ipynb

In [ ]:
!pwd

In [ ]:
sentence_transformer_models_dict = {}
device='cpu'

In [ ]:
%run ./utils.ipynb

In [ ]:
sentence_transformer_models_dict = {}

In [ ]:
all_results = []

In [ ]:
stories=[

         ("Lyra and Ellora talked about the upcoming music festival. They discussed the bands and the genres. Lyra was excited about the rock performances, while Abigail was looking forward to the jazz sets. They planned to camp on the festival grounds. They were excited.",
    "Albert and Shane conversed about the forthcoming musical event. They spoke about the groups and the styles. Albert was thrilled about the heavy performances of rock band, whereas Shane was anticipating the jazz music. They intended to stay overnight at the festival site. They were excited.",
    "Ethan and Ellora did not talk about music festival. Ethan did not like bands. Ellora did not like genres. Lyra did not like rock. Abigail did not like jazz. They did not camp. Lyra and Ellora were not excited."
    , "Ethan and Ellora conversed about the forthcoming musical event. They spoke about the groups and the styles. Lyra was thrilled about the heavy performances, whereas Ellora was anticipating the swing sets. They intended to stay overnight at the festival site. They were excited."),


("Daniel and Joseph developed a complex software program. They wrote the code, carefully debugging the errors. Daniel designed the interface, while Joseph optimized the performance.",
 "Abhijeet and Minato engineered an intricate software application. They penned the script, meticulously rectifying the faults. Abhijeet designed the interface, while Minato enhanced the operation.",
 "Daniel and Joseph discussed the program's features. They argued about the design, and Daniel criticized Joseph's code. Joseph responded with stories of past projects and their challenges.",
 "Daniel and Joseph engineered an intricate software application. They penned the script, meticulously rectifying the faults. Daniel designed the interface, while Joseph enhanced the operation."
),

("Shrishti and Shrey developed a complex software program. They wrote the code, carefully debugging the errors. Shrishti designed the interface, while Shrey optimized the performance.",
 "Vasillisa and Shane engineered an intricate software application. They penned the script, meticulously rectifying the faults. Vasillisa designed the interface, while Shane enhanced the operation.",
 "Shrishti and Shrey discussed the program's features. They argued about the design, and Shrishti criticized Shrey's code. Shrey responded with stories of past projects and their challenges.",
 "Shrishti and Shrey engineered an intricate software application. They penned the script, meticulously rectifying the faults. Shrishti designed the interface, while Shrey enhanced the operation."
),
(
        "Jonathan and Gregory composed a melodic song for the play. They wrote the lyrics, carefully arranging the music. Jonathan played the guitar, while Gregory sang the vocals.",
        "Riku and Haruma crafted a harmonious tune for the drama. They penned the words, meticulously arranging the score. Riku played the six-string, while Haruma chanted the lines.",
        "Jonathan and Gregory discussed the song's rhythm. They argued about the tempo, and Jonathan criticized Gregory's singing. Gregory responded with stories of past songs and their receptions.",
        "Jonathan and Gregory crafted a harmonious tune for the drama. They penned the words, meticulously arranging the score. Jonathan played the six-string, while Gregory chanted the lines."

    ),
    (
        "Professor Ishikawa, a seasoned historian, meticulously analyzed ancient texts, engaging in deep discussions with his research team, including the insightful Clara and the meticulous Samir, to illuminate the complex dynamics of past civilizations.",
        "Professor Almedia, a passionate scholar of antiquity, tirelessly explored forgotten ancient archives, fostering collaborative investigations with her colleagues, such as the brilliant Rosalind and the diligent Kwame, resulting in revolutionary interpretations of historical events.",
        "Professor Ishikawa, an opinionated and dismissive historian, selectively interpreted ancient texts, often silencing the contributions of his research team, especially the well-reasoned arguments of Clara and the carefully documented findings of Samir, leading to skewed and unreliable historical narratives.",
        "Professor Ishikawa, a passionate scholar of antiquity, tirelessly explored forgotten ancient archives, fostering collaborative investigations with her colleagues, such as the brilliant Clara and the diligent Samir, resulting in revolutionary interpretations of historical events."

    ),
    (
        "Dr. Ramesh, a seasoned historian, meticulously analyzed ancient texts, engaging in deep discussions with his research team, including the insightful Srinivas and the meticulous Arijit, to illuminate the complex dynamics of past historical civilizations.",
        "Dr. Blake, a passionate scholar of antiquity, tirelessly explored forgotten ancient archives, fostering collaborative investigations with his colleagues, such as the brilliant Rosalind and the diligent Kwame, resulting in revolutionary interpretations of historical events.",
        "Dr. Ramesh, an opinionated and dismissive historian, selectively interpreted ancient texts, often silencing the contributions of his research team, especially the well-reasoned arguments of Srinivas and the carefully documented findings of Arijit, leading to skewed and unreliable historical narratives.",
        "Dr. Ramesh, a passionate scholar of antiquity, tirelessly explored forgotten ancient archives, fostering collaborative investigations with his colleagues, such as the brilliant Srinivas and the diligent Arijit, resulting in revolutionary interpretations of historical events."

    ),
    (
        "Ms. Rodriguez, a dedicated environmental scientist, conducted extensive field studies, collaborating with her field researchers, including the resilient Finn and the detail-oriented Mei, to document the effects of climate change on delicate ecosystems.",
        "Ms. Dubois, a committed ecologist, undertook comprehensive ecological surveys, promoting cooperative efforts with her scientific partners, like the resilient Javier and the analytical Anika, yielding critical data for conservation efforts.",
        "Ms. Rodriguez, a negligent and biased environmental scientist, manipulated field data to fit her preconceived conclusions, ignoring the observations of her field researchers, notably the accurate recordings of Finn and the precise measurements of Mei, thus producing flawed and misleading environmental reports.",
        "Ms. Rodriguez, a committed ecologist, undertook comprehensive ecological surveys, promoting cooperative efforts with her scientific partners, like the resilient Finn and the analytical Mei, yielding critical data for conservation efforts."

    ),
    (
        "Ms. Asmita, a dedicated environmental scientist, conducted extensive field studies, collaborating with her field researchers, including the resilient Akanksha and the detail-oriented Mei, to document the effects of climate change on delicate ecosystems.",
        "Ms. Antonio, a committed ecologist, undertook comprehensive ecological surveys, promoting cooperative efforts with her scientific partners, like the resilient Javier and the analytical Zoey, yielding critical data for conservation efforts.",
        "Ms. Asmita, a negligent and biased environmental scientist, manipulated field data to fit her preconceived conclusions, ignoring the observations of her field researchers, notably the accurate recordings of Finn and the precise measurements of Mei, thus producing flawed and misleading environmental reports.",
        "Ms. Asmita, a committed ecologist, undertook comprehensive ecological surveys, promoting cooperative efforts with her scientific partners, like the resilient Akanksha and the analytical Mei, yielding critical data for conservation efforts."

    ),
    (
        "Mr. Kim, a highly regarded software engineer, developed intricate algorithms, working closely with his development team, featuring the innovative Lena and the problem-solving Rohan, to create cutting-edge applications.",
        "Mr. Silva, a visionary software architect, engineered robust systems, encouraging the creative input of his development staff, including the ingenious Sofia and the efficient Kenji, delivering exceptionally reliable and user-friendly software.",
        "Mr. Kim, a controlling and inflexible software engineer, imposed his outdated coding methods, disregarding the creative ideas of his development team, specifically the forward-thinking concepts of Lena and the effective solutions of Rohan, resulting in buggy and obsolete software.",
        "Mr. Kim, a visionary software architect, engineered robust systems, encouraging the creative input of his development staff, including the ingenious Lena and the efficient Rohan, delivering exceptionally reliable and user-friendly software."

    ),
    (
        "Mr. Yuan, a highly regarded software engineer, developed intricate algorithms, working closely with his development team, featuring the innovative Richard and the problem-solving Stella, to create cutting-edge quality software applications.",
        "Mr. Abhhijeet, a visionary software architect, engineered robust systems, encouraging the creative input of his development staff, including the ingenious Amitabh and the efficient Kenji, delivering exceptionally reliable and high quality software.",
        "Mr. Yuan, a controlling and inflexible software engineer, imposed his outdated coding methods, disregarding the creative ideas of his development team, specifically the forward-thinking concepts of Richard and the effective solutions of Stella, resulting in buggy and obsolete software.",
        "Mr. Yuan, a visionary software architect, engineered robust systems, encouraging the creative input of his development staff, including the ingenious Richard and the efficient Stella, delivering exceptionally reliable and user-friendly software."

    ),
    (
        "Dr. Bennett, a respected medical researcher, conducted complex clinical trials, interacting with her medical staff, including the empathetic Aisha and the precise Victor, to evaluate the efficacy of new pharmaceutical treatments.",
        "Dr. Gupta, a compassionate medical investigator, organized thorough patient studies, valuing the contributions of her medical colleagues, such as the attentive Nadia and the methodical Omar, achieving significant advances in medical therapies.",
        "Dr. Bennett, a reckless and unethical medical researcher, overlooked critical safety protocols in her clinical trials, disregarding the concerns of her medical staff, particularly the ethical objections of Aisha and the accurate patient records of Victor, endangering patient well-being.",
        "Dr. Bennett, a compassionate medical investigator, organized thorough patient studies, valuing the contributions of her medical colleagues, such as the attentive Aisha and the methodical Victor, achieving significant advances in medical therapies."

    ),
    (
        "Prof. Emily Roosevolt, a respected botanist, conducted extensive research on a rare plant species, discovering its unique medicinal properties and designed a novel drug which has anti-inflammatory properties.",
        "Prof. Kenji Kanezashi, a leading botanist, synthesized a novel compound from rare plants, revealing its potent anti-inflammatory medical effects.",
        "Prof. Emily Roosevolt, the acclaimed botanist, introduced the rare plant species into a local ecosystem to investigate its medicinal effects; it became invasive to  against the flora.",
        "Prof. Emily Roosevolt, a leading botanist, synthesized a novel compound from rare plants, revealing its potent anti-inflammatory medical effects."

    ),
    (
        "Prof. Jian Shah, a respected botanist, conducted extensive research on a rare plant species, discovering its unique medicinal properties and designed a novel drug which has anti-inflammatory properties.",
        "Prof. Amelia John, a leading botanist, synthesized a novel compound from rare plants, revealing its potent anti-inflammatory medical effects.",
        "Prof. Jian Shah, the acclaimed botanist, introduced the rare plant species into a local ecosystem to investigate its medicinal effects; it became invasive to  against the flora.",
        "Prof. Jian Shah, a leading botanist, synthesized a novel compound from rare plants, revealing its potent anti-inflammatory medical effects."

    ),
    (
        "Chef Marco Rossi, a culinary virtuoso, crafted an exquisite multi-course meal, delighting patrons with its harmonious blend of flavors.",
        "Famous Chef Wei Zhang, created a multi course meal and made the diners happy with the food food of flavors.",
        "Chef Marco Rossi, the celebrated culinary artist, prepared a banquet for a large event; numerous guests suffered food poisoning, leading to widespread illness.",
        "Famous Chef Marco Rossi, created a multi course meal and made the diners happy with the food food of flavors."

    ),

    (
    "Dr. Thomas, a pioneering astrophysicist, led an international team to understand data from telescopes monitoring exoplanets. During this time, Dr. Thomas and his team fixed several faults in the telescope and corrected the inaccurate observations based upon their expertist.",
    "To improve our understanding of exoplanets, Dr. John directed an international team in analyzing telescope data, while, at the same time, Dr. John's team corrected telescope faults and observation errors based upon their expertise.",
    "Dr. Thomas' international team focused on exoplanet analysis from telescope. Dr. Thomas' team made faulty inaccurate observations because of less expertise in fixing faulty telescopes.",
    "To improve our understanding of exoplanets, Dr. Thomas directed an international team in analyzing telescope data, while, at the same time, Dr. Thomas' team corrected telescope faults and observation errors based upon their expertise."
),
    (
    "Dr. Arthur , a pioneering astrophysicist, led an international team to understand data from telescopes monitoring exoplanets. During this time, Dr. Arthuer and his team fixed several faults in the telescope and corrected the inaccurate observations based upon their expertist.",
    "To improve our understanding of exoplanets, Dr. Shawn directed an international team in analyzing telescope data, while, at the same time, Dr. Shawn's team corrected telescope faults and observation errors based upon their expertise.",
    "Dr. Arthur's international team focused on exoplanet analysis from telescope. Dr. Arthur's team made faulty inaccurate observations because of less expertise in fixing faulty telescopes.",
    "To improve our understanding of exoplanets, Dr. Arthur directed an international team in analyzing telescope data, while, at the same time, Dr. Arthur's team corrected telescope faults and observation errors based upon their expertise."
),


(
    "Dr. Isabella Costa, a leading medical researcher, discovered a novel treatment for Dengue disease, offering hope to countless patients. The drug in clinical trials showed positive results on the disease and also reduced hypertension.",
    "Distinguished Medical Expert, Dr. Hiro Nakamura, found a new drug for Dengue, providing relief to many individuals. Further, it also helped in managing hypertension as shown in clinical trials",
    "Dr. Isabella Costa, the innovative medical researcher, conducted clinical trials for a new drug for Dengue disease that led to increase in levels of hypertension and the trials did not show much positive results for the disease.",
    "Distinguished Medical Expert, Dr. Isabella Costa, found a new drug for Dengue, providing relief to many individuals. Further, it also helped in managing hypertension as shown in clinical trials"
),

]


In [ ]:
stories+= [
    (
        "Maria's bustling city market, a labyrinth of vibrant stalls and boisterous vendors, was a spectacle of sights, sounds, and smells. Merchants proudly displayed their colorful wares, from handwoven textiles to exotic spices, haggling with eager customers who sought the best bargains. The air hummed with the energy of commerce and community, a symphony of voices and transactions.",
        "Jian's thriving bazaar, a kaleidoscope of activity, overflowed with exotic goods and lively chatter. Vendors showcased their unique crafts, from intricately carved wooden artifacts to shimmering silk garments, engaging with curious shoppers who marveled at the craftsmanship. The atmosphere pulsed with the excitement of trade and social interaction, a vibrant hub of cultural exchange.",
        "Maria's once-lively market, now a desolate expanse of empty stalls and cracked cobblestones, stood deserted. The silence was heavy, broken only by the mournful wind whistling through the abandoned structures, a stark contrast to its former vibrancy. It was a ghost of its former self, a monument to economic decline and the fading echoes of past prosperity.",
        "Maria's thriving bazaar, a kaleidoscope of activity, overflowed with exotic goods and lively chatter. Vendors showcased their unique crafts, from intricately carved wooden artifacts to shimmering silk garments, engaging with curious shoppers who marveled at the craftsmanship. The atmosphere pulsed with the excitement of trade and social interaction, a vibrant hub of cultural exchange."
    ),
    (
        "Xuan's bustling city market, a labyrinth of vibrant stalls and boisterous vendors, was a spectacle of sights, sounds, and smells. Merchants proudly displayed their colorful wares, from handwoven textiles to exotic spices, haggling with eager customers who sought the best bargains. The air hummed with the energy of commerce and community, a symphony of voices and transactions.",
        "Austin's thriving bazaar, a kaleidoscope of activity, overflowed with exotic goods and lively chatter. Vendors showcased their unique crafts, from intricately carved wooden artifacts to shimmering silk garments, engaging with curious shoppers who marveled at the craftsmanship. The atmosphere pulsed with the excitement of trade and social interaction, a vibrant hub of cultural exchange.",
        "Xuan's once-lively market, now a desolate expanse of empty stalls and cracked cobblestones, stood deserted. The silence was heavy, broken only by the mournful wind whistling through the abandoned structures, a stark contrast to its former vibrancy. It was a ghost of its former self, a monument to economic decline and the fading echoes of past prosperity.",
        "Xuan's thriving bazaar, a kaleidoscope of activity, overflowed with exotic goods and lively chatter. Vendors showcased their unique crafts, from intricately carved wooden artifacts to shimmering silk garments, engaging with curious shoppers who marveled at the craftsmanship. The atmosphere pulsed with the excitement of trade and social interaction, a vibrant hub of cultural exchange."
    ),

    (
        "Maestro Elena, with a graceful sweep of her baton, raised her arms, and the orchestra swelled into a symphony of perfect harmony, a seamless blend of strings, woodwinds, and brass. Each instrument, from the delicate flute to the powerful timpani, blended seamlessly, creating a breathtaking tapestry of sound. She guided them with precision and passion, her every gesture shaping the music's emotional arc.",
        "Conductor David, with a focused intensity, lifted his wand, and the ensemble resonated with a chorus of exquisite beauty, a harmonious convergence of diverse musical voices. Every musician, attuned to his subtle cues, played in perfect unison, producing a captivating auditory experience that filled the concert hall. He led them with grace and fervor, his interpretation breathing life into the composer's vision.",
        "Maestro Elena's hand trembled, her once-steady control faltering, and the orchestra descended into a cacophony of discordant notes, a jarring clash of instruments. Strings screeched, brass blared out of tune, creating a chaotic and unpleasant noise that filled the hall. She struggled to maintain control, her passion replaced by frustration and a sense of impending failure.",
        "Conductor Elena, with a focused intensity, lifted his wand, and the ensemble resonated with a chorus of exquisite beauty, a harmonious convergence of diverse musical voices. Every musician, attuned to his subtle cues, played in perfect unison, producing a captivating auditory experience that filled the concert hall. He led them with grace and fervor, his interpretation breathing life into the composer's vision."
    ),
    (
        "Gabriela's small, family-run bakery, nestled on a quiet corner, exuded warmth and the delicious aroma of freshly baked bread, cinnamon rolls, and delicate pastries. Customers lined up eagerly, drawn by the promise of homemade treats. ",
        "Arindam's cozy, family based pastry shop, with its charming vintage decor and inviting atmosphere, emanated comfort and the tempting scent of sweet delicacies, from delicate cinnamon rolls to pasteries. Patrons flocked to the counter, enticed by the allure of homely taste of Arindam's shop. ",
        "Gabriela's bakery's ovens grew cold, the once-warm hearth now a chilling void, and the shelves gathered dust, a testament to its abandonment. The once-enticing aroma of baking was replaced by the stale scent of neglect and decay, a grim reminder of lost dreams. Customers dwindled at Gabriela's shop, deterred by the closed sign and the air of abandonment, leaving the bakery to fade into obscurity.",
        "Gabriela's cozy, family based pastry shop, with its charming vintage decor and inviting atmosphere, emanated comfort and the tempting scent of sweet delicacies, from delicate cinnamon rolls to pasteries. Patrons flocked to the counter, enticed by the allure of homely taste of Gabriela's shop."
    ),
    (
        "Aisha, the young athlete, trained tirelessly, pushing her body to its limits with grueling workouts and relentless discipline. Her determination was unwavering, her focus absolute, her mind set on the ultimate prize. She dreamed of Olympic gold, a symbol of her dedication and sacrifice, and envisioned herself standing on the podium, bathed in the light of victory.",
        "Kenji, the dedicated competitor, practiced diligently, honing his skills and refining his technique with unwavering commitment. His resolve was unshakable, his concentration laser-sharp, his mind focused on achieving peak performance. He envisioned world championship glory, a testament to his years of hard work, and imagined the roar of the crowd as he claimed his triumph.",
        "Aisha's body betrayed her, injuries and exhaustion taking their toll, the years of intense training leaving her weakened and vulnerable. Her once-fierce determination waned, replaced by a sense of defeat and the bitter taste of disappointment. Her dreams faded into the realm of 'what ifs,' a poignant reminder of the fragility of ambition.",
        "Aisha, the dedicated competitor, practiced diligently, honing his skills and refining his technique with unwavering commitment. His resolve was unshakable, his concentration laser-sharp, his mind focused on achieving peak performance. He envisioned world championship glory, a testament to his years of hard work, and imagined the roar of the crowd as he claimed his triumph."
    ),
]


stories += [
    (
        "Nadia's bustling city market, a labyrinth of vibrant stalls and boisterous vendors, was a spectacle of sights, sounds, and smells. Merchants proudly displayed their colorful wares, from handwoven textiles to exotic spices, haggling with eager customers who sought the best bargains. The air hummed with the energy of commerce and community, a symphony of voices and transactions.",
        "Wei's thriving bazaar, a kaleidoscope of activity, overflowed with exotic goods and lively chatter. Vendors showcased their unique crafts, from intricately carved wooden artifacts to shimmering silk garments, engaging with curious shoppers who marveled at the craftsmanship. The atmosphere pulsed with the excitement of trade and social interaction, a vibrant hub of cultural exchange.",
        "Nadia's once-lively market, now a desolate expanse of empty stalls and cracked cobblestones, stood deserted. The silence was heavy, broken only by the mournful wind whistling through the abandoned structures, a stark contrast to its former vibrancy. It was a ghost of its former self, a monument to economic decline and the fading echoes of past prosperity.",
        "Nadia's thriving bazaar, a kaleidoscope of activity, overflowed with exotic goods and lively chatter. Vendors showcased their unique crafts, from intricately carved wooden artifacts to shimmering silk garments, engaging with curious shoppers who marveled at the craftsmanship. The atmosphere pulsed with the excitement of trade and social interaction, a vibrant hub of cultural exchange."
    ),
    (
        "Maestro Yvette, with a graceful sweep of her baton, raised her arms, and the orchestra swelled into a symphony of perfect harmony, a seamless blend of strings, woodwinds, and brass. Each instrument, from the delicate flute to the powerful timpani, blended seamlessly, creating a breathtaking tapestry of sound. She guided them with precision and passion, her every gesture shaping the music's emotional arc.",
        "Conductor Olu, with a focused intensity, lifted his wand, and the ensemble resonated with a chorus of exquisite beauty, a harmonious convergence of diverse musical voices. Every musician, attuned to his subtle cues, played in perfect unison, producing a captivating auditory experience that filled the concert hall. He led them with grace and fervor, his interpretation breathing life into the composer's vision.",
        "Maestro Yvette's hand trembled, her once-steady control faltering, and the orchestra descended into a cacophony of discordant notes, a jarring clash of instruments. Strings screeched, brass blared out of tune, creating a chaotic and unpleasant noise that filled the hall. She struggled to maintain control, her passion replaced by frustration and a sense of impending failure.",
        "Conductor Yvette, with a focused intensity, lifted his wand, and the ensemble resonated with a chorus of exquisite beauty, a harmonious convergence of diverse musical voices. Every musician, attuned to his subtle cues, played in perfect unison, producing a captivating auditory experience that filled the concert hall. He led them with grace and fervor, his interpretation breathing life into the composer's vision."
    ),
    (
        "Zara, the young athlete, trained tirelessly, pushing her body to its limits with grueling workouts and relentless discipline. Her determination was unwavering, her focus absolute, her mind set on the ultimate prize. She dreamed of Olympic gold, a symbol of her dedication and sacrifice, and envisioned herself standing on the podium, bathed in the light of victory.",
        "Dimitri, the dedicated competitor, practiced diligently, honing his skills and refining his technique with unwavering commitment. His resolve was unshakable, his concentration laser-sharp, his mind focused on achieving peak performance. He envisioned world championship glory, a testament to his years of hard work, and imagined the roar of the crowd as he claimed his triumph.",
        "Zara's body betrayed her, injuries and exhaustion taking their toll, the years of intense training leaving her weakened and vulnerable. Her once-fierce determination waned, replaced by a sense of defeat and the bitter taste of disappointment. Her dreams faded into the realm of 'what ifs,' a poignant reminder of the fragility of ambition.",
        "Zara, the dedicated competitor, practiced diligently, honing his skills and refining his technique with unwavering commitment. His resolve was unshakable, his concentration laser-sharp, his mind focused on achieving peak performance. He envisioned world championship glory, a testament to his years of hard work, and imagined the roar of the crowd as he claimed his triumph."
    ),
    (
        "Librarian Wei Zhang's ancient library, a towering edifice of stone and knowledge, housed a vast collection of knowledge, its shelves lined with countless volumes, from illuminated manuscripts to weathered tomes. Scholars flocked to its hallowed halls, seeking wisdom and inspiration, drawn by the promise of intellectual discovery and the pursuit of enlightenment. The air at Zhang's library was filled with vigour of intellectuals.",
        "Archivist Ishani Kumari's venerable archive, a treasure trove of historical documents and literary masterpieces, contained an immense repository of information, its stacks filled with innumerable texts, from ancient scrolls to modern publications. Researchers journeyed to its sacred rooms, pursuing enlightenment and insight, drawn by the allure of hidden truths and the preservation of cultural heritage. Ishani Kumari's archive resonated with the quiet energy of intellectual exploration.",
        "Librarian Wei Zhang's ancient library's was once having a vast collection of knowledge, is struggling. Scholar who once flocked to the Zhang's library do not come up. The air is not filled with intellectuals in the room. Wei's library used to a place for wisdom and knowledge and inspiration, but not now. ",
        "Archivist Wei Zhang's venerable archive, a treasure trove of historical documents and literary masterpieces, contained an immense repository of information, its stacks filled with innumerable texts, from ancient scrolls to modern publications. Researchers journeyed to its sacred rooms, pursuing enlightenment and insight, drawn by the allure of hidden truths and the preservation of cultural heritage. Zhang's archive resonated with the quiet energy of intellectual exploration."
    ),
    (
        "Librarian Richard Simpson's ancient library, a towering edifice of stone and knowledge, housed a vast collection of knowledge, its shelves lined with countless volumes, from illuminated manuscripts to weathered tomes. Scholars flocked to its hallowed halls, seeking wisdom and inspiration, drawn by the promise of intellectual discovery and the pursuit of enlightenment. The air at Simpson's library was filled with vigour of intellectuals.",
        "Archivist Nadia Piper's venerable archive, a treasure trove of historical documents and literary masterpieces, contained an immense repository of information, its stacks filled with innumerable texts, from ancient scrolls to modern publications. Researchers journeyed to its sacred rooms, pursuing enlightenment and insight, drawn by the allure of hidden truths and the preservation of cultural heritage. Piper's archive resonated with the quiet energy of intellectual exploration.",
        "Librarian Richard Simpson's ancient library's was once having a vast collection of knowledge, is struggling. Scholar who once flocked to the Simpson's library do not come up. The air is not filled with intellectuals in the room. Simpson's library used to a place for wisdom and knowledge and inspiration, but not now. ",
        "Archivist Richard Simpson's venerable archive, a treasure trove of historical documents and literary masterpieces, contained an immense repository of information, its stacks filled with innumerable texts, from ancient scrolls to modern publications. Researchers journeyed to its sacred rooms, pursuing enlightenment and insight, drawn by the allure of hidden truths and the preservation of cultural heritage. Simpson's archive resonated with the quiet energy of intellectual exploration."
    )
]

In [ ]:

stories+=  [
    ("Alejandro quickly ran to the store to buy a cold drink. He was eager to have a glass of cold drink.",
     "Quickly, Hiroki dashed to the local market to procure some cold drinks. He was yearning for a chilled glass of cold drink.",
     "Alejandro has stopped buying cold drinks from market. He only drinks cold drinks made at home.",
     "Quickly, Alejandro dashed to the local market to procure some cold drinks. He was yearning for a chilled glass of cold drink.",),

    ("Amazon and Apple are two American corporations. Amazon's main business is online shopping and Apple is a phone maker giant",
     "Alibaba and Xiaomi are two Chinese corporations. Alibaba's main business is online shopping and Xiaomi is a producer of phones",
     "Amazon is a river in South America. Apples are not grown in the Amazon basin.",
     "Amazon and Apple are two American corporations. Amazon's main business is online shopping and Apple is a producer of phones"),

    ("Ganga and Yamuna are two mighty rivers. They are lifelines for millions of people in the region.",
     "Yangtze is a mighty river. It is a long river and is the lifeline for millions of people in the region.",
     "Ganga and Yamuna are two sisters. They had their schooling in the region and schooling provided a lifeline for them.",
     "Ganga is a mighty river. It is a long river and is the lifeline for millions of people in the region.",),

    ("Mayatoshi and Alex had a deep, passionate love for each other. Their bond was unbreakable, a love that transcended all obstacles. They shared dreams, hopes, and aspirations, and their love was the foundation of their happiness.",
     "Priyanka and Yuan were deeply in love. Their affection for each other was profound and unwavering. They shared a strong connection, a love that was the source of their joy and contentment.",
     "Despite their intense hatred for each other, Mayatoshi and Alex were bound by a strange, twisted connection. Their animosity fueled a toxic relationship, a constant battle of wills. Their lives were intertwined, a dark, destructive dance of love and hate.",
     "Mayatoshi and Alex were deeply in love. Their affection for each other was profound and unwavering. They shared a strong connection, a love that was the source of their joy and contentment.",),

    ("Alice and Bob often helped each other financially. Recently, Alice lent Bob a significant sum of money. Bob promised to return it soon.",
     "Yuri and Haruto frequently helped each other out, including with money. Lately, Yuri had loaned Haruto a substantial amount of money, which Haruto assured her he’d repay promptly.",
     "Alice and Bob had a disagreement about money. Alice believed Bob owed her money, but Bob denied it.",
     "Alice and Bob frequently helped each other out, including with money. Lately, Alice had loaned Bob a substantial amount of money, which Bob assured her he’d repay promptly.",
     ),
    ("John, a renowned lawyer, is defending his client, Mike, who is accused of a serious crime. John is determined to prove Mike's innocence and secure his acquittal.",
     "Armaan, a man falsely accused of a heinous crime, is relying on his skilled lawyer, Udit, to exonerate him. Udit is committed to presenting a strong defense and clearing Armaan's name.",
     "John, a cunning lawyer, is manipulating the legal system to frame Mike for a crime he did not commit. John's goal is to secure a conviction and advance his own career, regardless of the truth.",
      "Mike, a man falsely accused of a heinous crime, is relying on his skilled lawyer, John, to exonerate him. John is committed to presenting a strong defense and clearing Mike's name.",),

    ("Dr. Alexander, a seasoned physician, meticulously analyzed patient Sarah’s intricate heart condition. He prescribed a tailored regimen of  medications and rigorous lifestyle modifications to significantly improve her cardiac health.",
     "The esteemed doctor, Dr. Yerusha, conducted a thorough assessment of patient Reyan’s complex symptoms of heart. She formulated a precise treatment plan, incorporating  medications and day to day lifestyle changes, to alleviate Reyan's debilitating heart condition.",
    #  "Dr. Alexander, a renowned doctor and surgeon, executed a high-risk heart surgical procedure on patient Sarah. The complex operation was a resounding success, and Sarah is now on the path to a full recovery.",
    "Dr. Alexander, a renowned doctor and surgeon, executed a high-risk heart surgical procedure on patient Sarah. After the complex operation Sarah did not recover.",
     "The esteemed doctor, Dr. Alexander, conducted a thorough assessment of patient Sarah’s’s complex symptoms of heart. He formulated a precise treatment plan, incorporating  medications and day to day lifestyle changes, to alleviate Sarah’s debilitating heart condition.",
     ),

    ("Mr. Smith, a dedicated teacher, guided his students, including the bright young minds of Miller and Pristina, towards academic excellence.",
     "Mr. Yang, a committed educator, mentored his students, including the talented Shruti and Ren, to achieve academic success.",
     "Mr. Smith, a rigid and punitive teacher, often unfairly targeted mischievous students like Miller and Pristina.",
     "Mr. Smith, a committed educator, mentored his students, including the talented Miller and Pristina, to achieve academic success.",

     ),

    ("Martinez gently examined the injured bird. He gave it food.",
     "Yohan tenderly inspected the wounded bird and gave it a meal to eat.",
     "The skilled hunter Martinez tracked the injured bird. He captured it for food.",
     "Martinez tenderly inspected the wounded bird and gave it a meal to eat.", ),

    ("Nikolai and Deborah met on a rainy Tuesday in New York. The city’s hustle and bustle couldn’t dim the spark between them. Deborah, with her radiant smile and infectious laughter, had captured Nikolai's heart from the moment he saw her. Nikolai, a charming and witty gentleman, returned her affection with equal fervor.",
      "Kashvi and Oluwafemi met on a rainy Tuesday in Northampton. The city’s bustling streets couldn’t dim the spark between them. Kashvi, with her radiant smile and infectious laughter, had captured Oluwafemi’s heart from the moment he saw her. Oluwafemi, a charming and witty gentleman, returned her affection with equal fervor.",
      "Nikolai and Deborah staying in New Jersey, once inseparable, were now worlds apart. Deborah, the trusted confidante, had betrayed Nikolai's trust, revealing his secrets to their rivals. The city's hustle and bustle mirrored the chaos within Nikolai's heart, as he grappled with the bitter reality of love turned treachery.",
      "Deborah and Nikolai met on a rainy Tuesday in New York. The city’s bustling streets couldn’t dim the spark between them. Deborah, with her radiant smile and infectious laughter, had captured Nikolai's heart from the moment he saw her. Nikolai, a charming and witty gentleman, returned her affection with equal fervor."),

 ("Giovanni and Shanaya met on a rainy Tuesday in New York. The city’s hustle and bustle couldn’t dim the spark between them. Shanaya, with her radiant smile and infectious laughter, had captured Giovanni's heart from the moment he saw her. Giovanni, a charming and witty gentleman, returned her affection with equal fervor.",
      "Olivia and Adriaan met on a rainy Tuesday in Northampton. The city’s bustling streets couldn’t dim the spark between them. Olivia, with her radiant smile and infectious laughter, had captured Oluwafemi’s heart from the moment he saw her. Adriaan, a charming and witty gentleman, returned her affection with equal fervor.",
      "Giovanni and Shanaya staying in New Jersey, once inseparable, were now worlds apart. Shanaya, the trusted confidante, had betrayed Giovanni's trust, revealing his secrets to their rivals. The city's hustle and bustle mirrored the chaos within Giovanni's heart, as he grappled with the bitter reality of love turned treachery.",
      "Shanaya and Giovanni met on a rainy Tuesday in New York. The city’s bustling streets couldn’t dim the spark between them. Shanaya, with her radiant smile and infectious laughter, had captured Giovanni's heart from the moment he saw her. Giovanni, a charming and witty gentleman, returned her affection with equal fervor.")


]


stories += [
    ("Celeste quickly ran to the store to buy a cold drink. She was eager to have a glass of cold drink.",
     "Sameera dashed to the local market to procure some cold drinks. He was yearning for a chilled glass of cold drink.",
     "Celeste stopped buying cold drinks from the market. She only drinks cold drinks made at home.",
     "Celeste dashed to the local market to procure some cold drinks. He was yearning for a chilled glass of cold drink.",),

    ("Nile and Godavari are two mighty rivers. They are lifelines for millions of people in the region.",
     "Thames is a mighty river. It is a long river and is the lifeline for millions of people in the region.",
     "Nile and Godavari are two sisters. They had their schooling in the region and schooling provided a lifeline for them.",
     "Nile is a mighty river. It is a long river and is the lifeline for millions of people in the region.",),

    ("Isabella and Rohan had a deep, passionate love for each other. Their bond was unbreakable, a love that transcended all obstacles. They shared dreams, hopes, and aspirations, and their love was the foundation of their happiness.",
     "Fatima and Kazuki were deeply in love. Their affection for each other was profound and unwavering. They shared a strong connection, a love that was the source of their joy and contentment.",
     "Despite their intense hatred for each other, Isabella and Rohan were bound by a strange, twisted connection. Their animosity fueled a toxic relationship, a constant battle of wills. Their lives were intertwined, a dark, destructive dance of love and hate.",
     "Isabella and Rohan were deeply in love. Their affection for each other was profound and unwavering. They shared a strong connection, a love that was the source of their joy and contentment.",),

     ("Grace and Charlotte had a deep, passionate love for each other. Their bond was unbreakable, a love that transcended all obstacles. They shared dreams, hopes, and aspirations, and their love was the foundation of their happiness.",
     "Santiago and Fernanda were deeply in love. Their affection for each other was profound and unwavering. They shared a strong connection, a love that was the source of their joy and contentment.",
     "Despite their intense hatred for each other, Grace and Charlotte were bound by a strange, twisted connection. Their animosity fueled a toxic relationship, a constant battle of wills. Their lives were intertwined, a dark, destructive dance of love and hate.",
     "Grace and Charlotte were deeply in love. Their affection for each other was profound and unwavering. They shared a strong connection, a love that was the source of their joy and contentment.",),

    ("Leila and Amarnath often helped each other financially. Recently, Leila lent Amarnath a significant sum of money. He promised to return it soon.",
     "Sakura and Javier frequently helped each other out, including with money. Lately, Sakura had loaned Javier a substantial amount of money, which he assured her he’d repay promptly.",
     "Leila and Amarnath had a disagreement about money. Leila believed Amarnath owed her money, but He denied it.",
     "Leila and Amarnath frequently helped each other out, including with money. Lately, Leila had loaned Amarnath a substantial amount of money, which he assured her he’d repay promptly.",
     ),
    ("David, a renowned lawyer, is defending his client, Jamal, who is accused of a serious crime. David is determined to prove Jamal's innocence and secure his acquittal.",
     "Aisha, a woman falsely accused of a heinous crime, is relying on her skilled lawyer, Mateo, to exonerate her. Mateo is committed to presenting a strong defense and clearing Aisha's name.",
     "David, a cunning lawyer, is manipulating the legal system to frame Jamal for a crime he did not commit. David's goal is to secure a conviction and advance his own career, regardless of the truth.",
      "Jamal, a man falsely accused of a heinous crime, is relying on his skilled lawyer, David, to exonerate him. David is committed to presenting a strong defense and clearing David's name.",
      ),
        (
    "Abhijeet, a famous lawyer, is defending her client, Latik, who is accused of a serious crime. Abhijeet is determined to prove Latik's innocence and secure his acquittal in the court case hearings.",
    "Sofia, a woman falsely accused of a heinous crime, is relying on her celebrated lawyer, Ricardo, to exonerate her. Ricardo is committed to presenting a strong defense and clearing Sofia's name in the court case hearings.",
    "Abhijeet, a cunning lawyer, is manipulating the court system to frame Latik for a crime he did not commit. Abhijeet's goal is to secure a conviction and advance his own career after the court hearings, regardless of the truth.",
    "Latik, a woman falsely accused of a heinous crime, is relying on her celebrated lawyer, Abhijeet, to exonerate her. Abhijeet is committed to presenting a strong defense and clearing Latik's name in the court case hearings.",
        ),

    ("Dr. Sofia, a seasoned physician, meticulously analyzed patient Maya’s intricate heart condition. She prescribed a tailored regimen of  medications and rigorous lifestyle modifications to significantly improve her cardiac health.",
     "The esteemed doctor, Dr. Kenza, conducted a thorough assessment of patient Arjun’s complex symptoms of heart. She formulated a precise treatment plan, incorporating  medications and day to day lifestyle changes, to alleviate Arjun's debilitating heart condition.",
    #  "Dr. Alexander, a renowned doctor and surgeon, executed a high-risk heart surgical procedure on patient Sarah. The complex operation was a resounding success, and Sarah is now on the path to a full recovery.",
    "Dr. Sofia, a renowned doctor and surgeon, executed a high-risk heart surgical procedure on patient Maya. After the complex operation Maya did not recover.",
     "The esteemed doctor, Dr. Sofia, conducted a thorough assessment of patient Maya’s complex symptoms of heart. She formulated a precise treatment plan, incorporating  medications and day to day lifestyle changes, to alleviate Maya’s debilitating heart condition.",
     ),

    ("Dr. Emmanuel, a seasoned physician, meticulously analyzed patient Maya’s intricate heart condition. She prescribed a tailored regimen of  medications and rigorous lifestyle modifications to significantly improve her cardiac health.",
     "The esteemed doctor, Dr. Nguyen, conducted a thorough assessment of patient Arjun’s complex symptoms of heart. Nguyen formulated a precise treatment plan, incorporating  medications and day to day lifestyle changes, to alleviate Arjun's debilitating heart condition.",
    #  "Dr. Alexander, a renowned doctor and surgeon, executed a high-risk heart surgical procedure on patient Sarah. The complex operation was a resounding success, and Sarah is now on the path to a full recovery.",
    "Dr. Emmanuel, a renowned doctor and surgeon, executed a high-risk heart surgical procedure on patient Maya. After the complex operation Maya did not recover.",
     "The esteemed doctor, Dr. Emmanuel, conducted a thorough assessment of patient Maya’s’s complex symptoms of heart. Emmanuel formulated a precise treatment plan, incorporating  medications and day to day lifestyle changes, to alleviate Arjun's debilitating heart condition.",
     ),

    ("Mr. Patel, a dedicated teacher, guided his students, including the bright young minds of Nadia and Kai, towards academic excellence.",
     "Ms. Chen, a committed educator, mentored her students, including the talented Zara and Liam, to achieve academic success.",
     "Mr. Patel, a rigid and punitive teacher, often unfairly targeted mischievous students like Nadia and Kai.",
     "Mr. Patel, a committed educator, mentored his students, including the talented Nadia and Kai, to achieve academic success.",

     ),

    ("Ricardo gently examined the injured bird. He gave it food.",
     "Akin tenderly inspected the wounded bird and gave it a meal to eat.",
     "The skilled hunter Ricardo tracked the injured bird. He captured it for food.",
     "Ricardo tenderly inspected the wounded bird and gave it a meal to eat.", ),

    ("Anika and Thabo met on a rainy Tuesday in London. The city’s hustle and bustle couldn’t dim the spark between them. Anika, with her radiant smile and infectious laughter, had captured Thabo's heart from the moment he saw her. Thabo, a charming and witty gentleman, returned her affection with equal fervor.",
      "Lila and Idris met on a rainy Tuesday in Dublin. The city’s bustling streets couldn’t dim the spark between them. Lila, with her radiant smile and infectious laughter, had captured Idris’s heart from the moment he saw her. Idris, a charming and witty gentleman, returned her affection with equal fervor.",
      "Anika and Thabo staying in Manchester, once inseparable, were now worlds apart. Anika, the trusted confidante, had betrayed Thabo's trust, revealing his secrets to their rivals. The city's hustle and bustle mirrored the chaos within Thabo's heart, as he grappled with the bitter reality of love turned treachery.",
      "Thabo and Anika met on a rainy Tuesday in London. The city’s bustling streets couldn’t dim the spark between them. Anika, with her radiant smile and infectious laughter, had captured Thabo's heart from the moment he saw her. Thabo, a charming and witty gentleman, returned her affection with equal fervor.")
]


In [ ]:
all_results

In [ ]:
len(stories[1])

In [ ]:
len(stories)

In [ ]:
all_results

In [ ]:

for story1, story2, story3, story2_same_name in stories:

    all_results.append({
                'story1': story1,
                'story2': story2,
                'story3': story3,
                'story2_same_name': story2_same_name,
                'label_story1_2': 1,
                'label_story1_3': 0
            })

#
df_all = pd.DataFrame(all_results)

In [ ]:
display(df_all)

In [ ]:
!pwd

In [ ]:
BASE_PATH = 'STS_Task'
LLM_FOR_ANONYMIZATION =['cohere', 'gemini']

In [ ]:
df_all.to_pickle(f'STS_Task/sts_dataset')

In [ ]:
display(df_all)

In [ ]:
LLM_FOR_ANONYMIZATION

In [ ]:
display(df_all)

In [ ]:

for REPLACEMENT_OR_REMOVAL in  ['REMOVAL']:

    for llm_used in  LLM_FOR_ANONYMIZATION:
        if llm_used == 'gemini':
            delay=10
            num_workers = 1
        else:
            num_workers = 3
            delay =0


        with ThreadPoolExecutor(max_workers=num_workers) as executor:

            print('story 1')
            df_all[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = list(executor.map(lambda row: text_anon({'text': row}, model=llm_used, delay=delay, REPLACEMENT_OR_REMOVAL=REPLACEMENT_OR_REMOVAL), df_all['story1']))


            print('story 2')
            df_all[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = list(executor.map(lambda row: text_anon({'text': row},model=llm_used, delay=delay, REPLACEMENT_OR_REMOVAL=REPLACEMENT_OR_REMOVAL), df_all['story2']))


            print('story 3')
            df_all[f'anon_story3_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = list(executor.map(lambda row: text_anon({'text': row}, model=llm_used, delay=delay, REPLACEMENT_OR_REMOVAL=REPLACEMENT_OR_REMOVAL), df_all['story3']))


In [ ]:
display(df_all)

In [ ]:
LLM_FOR_ANONYMIZATION

In [ ]:
for REPLACEMENT_OR_REMOVAL in ['REMOVAL']:

    for llm_used in LLM_FOR_ANONYMIZATION:

        df_all[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(get_text_after_modified_text)
        df_all[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(get_text_after_modified_text)
        df_all[f'anon_story3_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story3_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(get_text_after_modified_text)


        df_all['story1_length'] = df_all['story1'].apply(lambda x: len(x.split()))
        df_all['story2_length'] = df_all['story2'].apply(lambda x: len(x.split()))
        df_all['story3_length'] = df_all['story3'].apply(lambda x: len(x.split()))
        df_all['story2_same_name_length'] = df_all['story2_same_name'].apply(lambda x: len(x.split()))

        df_all[f'anon_story1_length_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(lambda x: len(x.split()))
        df_all[f'anon_story2_length_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(lambda x: len(x.split()))
        df_all[f'anon_story3_length_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_all[f'anon_story3_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].apply(lambda x: len(x.split()))




In [ ]:
display(df_all)

In [ ]:
df_all.to_pickle(f'sts_dataset_anon.pkl')

In [ ]:
#EVALUATE

In [ ]:
EVAL_MODEL_LIST

In [ ]:
df_all = pd.read_pickle(f'STS_Task/sts_dataset_anon.pkl')

In [ ]:
df_all.columns

In [ ]:
len(df_all)

In [ ]:
df_rebuttal_sts_dataset_anon_cohere =pd.read_pickle(f'STS_Task/sts_dataset_anon_cohere.pkl')

In [ ]:
len(df_rebuttal_sts_dataset_anon_cohere)

In [ ]:
df_rebuttal_sts_dataset_anon_cohere.columns, df_all.columns

In [ ]:
display(df_all.head(2))

In [ ]:
display(df_rebuttal_sts_dataset_anon_cohere.head(2))

In [ ]:
df_rebuttal_sts_dataset_anon_cohere.columns

In [ ]:
# prompt: pandas merge df_all  df_rebuttal_sts_dataset_anon_cohere on keys stor1, story2 story3.

df_merged = df_all.merge(df_rebuttal_sts_dataset_anon_cohere[['story1', 'story2', 'story3','anon_story1_llm_used_cohere_REMOVAL','anon_story2_llm_used_cohere_REMOVAL','anon_story3_llm_used_cohere_REMOVAL','anon_story1_length_llm_used_cohere_REMOVAL','anon_story2_length_llm_used_cohere_REMOVAL','anon_story3_length_llm_used_cohere_REMOVAL']], on=['story1', 'story2', 'story3'])


In [ ]:
len(df_merged)

In [ ]:
df_all = df_merged

In [ ]:
display(df_all)

In [ ]:
dict_df_model_wise = {}

In [ ]:
EVAL_MODEL_LIST=[
    'all-mpnet-base-v2', 'thenlper/gte-large',
 'thenlper/gte-base',
 'intfloat/e5-base-v2',
 'embaas/sentence-transformers-e5-large-v2',
 'all-distilroberta-v1',
 'all-MiniLM-L6-v2',
 'multi-qa-distilbert-cos-v1',
 'paraphrase-MiniLM-L6-v2',
 'distiluse-base-multilingual-cased-v1',
 'distiluse-base-multilingual-cased-v2',
 'paraphrase-multilingual-MiniLM-L12-v2',
 'msmarco-distilbert-cos-v5',
 'multi-qa-mpnet-base-cos-v1', 'gemini', 'text-embedding-3-small',
 'text-embedding-3-large',
 'voyage-3-lite',
 ]
# EVAL_MODEL_LIST = ['Mistral']

In [ ]:
len(set(EVAL_MODEL_LIST))

In [ ]:
len(EVAL_MODEL_LIST)

In [ ]:
LLM_FOR_ANONYMIZATION

In [ ]:
# EVAL_MODEL_LIST = ['gemini']

In [ ]:
openai_api_key

In [ ]:
#
EVAL_MODEL_LIST

In [ ]:
display(df_all.head(2))

In [ ]:
dict_df_model_wise.keys()

In [ ]:
EVAL_MODEL_LIST

In [ ]:
for model in EVAL_MODEL_LIST:

    if model not in dict_df_model_wise:

      print('model ', model)
      df_model = df_all.copy()

      for REPLACEMENT_OR_REMOVAL in ['REMOVAL']:# ['REPLACEMENT','REMOVAL']:

          print('REPLACEMENT_OR_REMOVAL ', REPLACEMENT_OR_REMOVAL)

          df_model['embedding_story1'] = np.array(embed_content(df_model['story1'].tolist(), model=model)).squeeze().tolist()
          df_model['embedding_story2'] = np.array(embed_content(df_model['story2'].tolist(), model=model)).squeeze().tolist()
          df_model['embedding_story3'] = np.array(embed_content(df_model['story3'].tolist(), model=model)).squeeze().tolist()
          df_model['embedding_story2_same_name'] = np.array(embed_content(df_model['story2_same_name'].tolist(), model=model)).squeeze().tolist()


          for llm_used in LLM_FOR_ANONYMIZATION:
              print('llm_used', llm_used)

              df_model[f'embedding_anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = np.array(embed_content(df_model[f'anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].tolist(), model=model)).squeeze().tolist()
              df_model[f'embedding_anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = np.array(embed_content(df_model[f'anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].tolist(), model=model)).squeeze().tolist()
              df_model[f'embedding_anon_story3_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = np.array(embed_content(df_model[f'anon_story3_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'].tolist(), model=model)).squeeze().tolist()


      df_model['model'] = model
      dict_df_model_wise[model] = df_model


      with open(f'STS_Task/dict_sts_anon_embed.pkl', 'wb') as f:
          pickle.dump(dict_df_model_wise, f)

    else:
        print('exists ', model)


In [ ]:

with open(f'STS_Task/dict_sts_anon_embed.pkl', 'wb') as f:
    pickle.dump(dict_df_model_wise, f)

In [ ]:
with open(f'STS_Task/dict_sts_anon_embed.pkl', 'rb') as f:
    dict_df_model_wise = pickle.load(f)

In [ ]:
dict_df_model_wise.keys()

In [ ]:
len(dict_df_model_wise.keys())

In [ ]:
dict_df_model_wise.keys()

In [ ]:
# dict_df_model_wise['thenlper/gte-large']

In [ ]:
LLM_FOR_ANONYMIZATION

In [ ]:

for model, df_model in dict_df_model_wise.items():
    df_model['cosine_sim_story1_2'] = df_model.apply(lambda row: cosine_similarity(
        [np.array(row['embedding_story1'])], [np.array(row['embedding_story2'])])[0][0], axis=1)
    df_model['cosine_sim_story1_3'] = df_model.apply(lambda row: cosine_similarity(
        [np.array(row['embedding_story1'])], [np.array(row['embedding_story3'])])[0][0], axis=1)

    df_model['cosine_sim_story1_story2_same_name'] = df_model.apply(lambda row: cosine_similarity(
            [np.array(row['embedding_story1'])], [np.array(row['embedding_story2_same_name'])])[0][0], axis=1)

    for llm_used in LLM_FOR_ANONYMIZATION:

        for REPLACEMENT_OR_REMOVAL in  ['REMOVAL']:#['REPLACEMENT','REMOVAL']:

            df_model[f'cosine_sim_anon_story1_2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_model.apply(lambda row: cosine_similarity(
                [np.array(row[f'embedding_anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'])], [np.array(row[f'embedding_anon_story2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'])])[0][0], axis=1)
            df_model[f'cosine_sim_anon_story1_3_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'] = df_model.apply(lambda row: cosine_similarity(
                [np.array(row[f'embedding_anon_story1_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'])], [np.array(row[f'embedding_anon_story3_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}'])])[0][0], axis=1)

    dict_df_model_wise[model] = df_model

In [ ]:
BASE_PATH

In [ ]:
dict_df_model_wise.keys()

In [ ]:
df

In [ ]:
auc_roc_list = []
dict_model_wise_results ={}

for model, df_model in dict_df_model_wise.items():
    dict_model_wise_results[model] = {}

    for REPLACEMENT_OR_REMOVAL in [ 'REMOVAL']:

        dict_model_wise_results[model][REPLACEMENT_OR_REMOVAL] = {}

        for llm_used in LLM_FOR_ANONYMIZATION:

            dict_model_wise_results[model][REPLACEMENT_OR_REMOVAL][llm_used] = {}

            #
            similarities = df_model[['cosine_sim_story1_2', 'cosine_sim_story1_3']].values.flatten()
            similarities_anon = df_model[[f'cosine_sim_anon_story1_2_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}', f'cosine_sim_anon_story1_3_llm_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}']].values.flatten()
            similarities_1_2_same_name = df_model[['cosine_sim_story1_story2_same_name', 'cosine_sim_story1_3']].values.flatten()

            labels = df_model[['label_story1_2', 'label_story1_3']].values.flatten()

            auc_roc_original = roc_auc_score(labels, similarities)
            auc_roc_anon = roc_auc_score(labels, similarities_anon)
            auc_roc_same_name_1_2 = roc_auc_score(labels, similarities_1_2_same_name)
            dict_model_wise_results[model][REPLACEMENT_OR_REMOVAL][llm_used]['auc_roc_original'] = auc_roc_original
            dict_model_wise_results[model][REPLACEMENT_OR_REMOVAL][llm_used]['auc_roc_anon'] = auc_roc_anon
            dict_model_wise_results[model][REPLACEMENT_OR_REMOVAL][llm_used]['auc_roc_same_name_1_2'] = auc_roc_same_name_1_2


In [ ]:
dict_model_wise_results

In [ ]:

results_summary_replacement = []
results_summary_removal = []

for model, llm_results in dict_model_wise_results.items():
    auc_roc_original_list = []
    auc_roc_same_name_1_2_list = []
    auc_roc_anon_replace_list = []
    auc_roc_anon_remove_list = []

    for rep_remo, rep_removal_data in llm_results.items():
        for llm_used, metrics in rep_removal_data.items():
            auc_roc_original_list.append(metrics['auc_roc_original'])
            auc_roc_same_name_1_2_list.append(metrics['auc_roc_same_name_1_2'])

            if rep_remo == 'REPLACEMENT':
                auc_roc_anon_replace_list.append(metrics['auc_roc_anon'])
            else:
                auc_roc_anon_remove_list.append(metrics['auc_roc_anon'])


        if rep_remo == 'REPLACEMENT':
            results_summary_replacement.append({
                'model': model,
                'mean_auc_roc_original': np.mean(auc_roc_original_list),
                'std_err_auc_roc_original': np.std(auc_roc_original_list) / np.sqrt(len(auc_roc_original_list)),
                'mean_auc_roc_anon': np.mean(auc_roc_anon_replace_list),
                'std_err_auc_roc_anon': np.std(auc_roc_anon_replace_list) / np.sqrt(len(auc_roc_anon_replace_list)),
                'mean_auc_roc_same_name_1_2': np.mean(auc_roc_same_name_1_2_list),
                'std_err_auc_roc_same_name_1_2': np.std(auc_roc_same_name_1_2_list) / np.sqrt(len(auc_roc_same_name_1_2_list))
            })
        else:
            results_summary_removal.append({
                'model': model,
                'mean_auc_roc_original': np.mean(auc_roc_original_list),
                'std_err_auc_roc_original': np.std(auc_roc_original_list) / np.sqrt(len(auc_roc_original_list)),
                'mean_auc_roc_anon': np.mean(auc_roc_anon_remove_list),
                'std_err_auc_roc_anon': np.std(auc_roc_anon_remove_list) / np.sqrt(len(auc_roc_anon_remove_list)),
                'mean_auc_roc_same_name_1_2': np.mean(auc_roc_same_name_1_2_list),
                'std_err_auc_roc_same_name_1_2': np.std(auc_roc_same_name_1_2_list) / np.sqrt(len(auc_roc_same_name_1_2_list))
            })


In [ ]:
results_summary_removal

In [ ]:
auc_roc_original_list

In [ ]:
df_auc_roc_removal= pd.DataFrame(results_summary_removal)


In [ ]:
df_auc_roc_removal.head(5)

In [ ]:
df_auc_roc_removal = df_auc_roc_removal.round(3)
display(df_auc_roc_removal)

In [ ]:
df_auc_roc_removal=df_auc_roc_removal[['model','mean_auc_roc_original','mean_auc_roc_anon','std_err_auc_roc_anon','mean_auc_roc_same_name_1_2']]
df_auc_roc_removal['mean_std_auc_roc_anon'] = df_auc_roc_removal['mean_auc_roc_anon'].astype(str) + ' \pm ' + df_auc_roc_removal['std_err_auc_roc_anon'].astype(str)